In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load dataset
dataset = pd.read_csv('aurangabad_lstm_10.csv')
data = dataset.values
data = data.astype('float32')

In [3]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

In [4]:
# Split data into training and testing sets
train_size = int(len(data) * 0.67)
test_size = len(data) - train_size
train, test = data[0:train_size,:], data[train_size:len(data),:]

In [5]:
# Create function to create LSTM model
def create_model():
    model = Sequential()
    model.add(LSTM(50, input_shape=(1, 1)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [6]:
# Reshape input data to be 3-dimensional
X_train = np.reshape(train, (train.shape[0], 1, train.shape[1]))
X_test = np.reshape(test, (test.shape[0], 1, test.shape[1]))

In [7]:
# Create and fit the LSTM model
model = create_model()
model.fit(X_train, train, epochs=50, batch_size=1, verbose=2)

Epoch 1/50
1021/1021 - 9s - loss: 9.3737e-04 - 9s/epoch - 9ms/step
Epoch 2/50
1021/1021 - 5s - loss: 9.4639e-05 - 5s/epoch - 5ms/step
Epoch 3/50
1021/1021 - 5s - loss: 3.8127e-05 - 5s/epoch - 5ms/step
Epoch 4/50
1021/1021 - 5s - loss: 4.4851e-05 - 5s/epoch - 5ms/step
Epoch 5/50
1021/1021 - 5s - loss: 2.7354e-05 - 5s/epoch - 5ms/step
Epoch 6/50
1021/1021 - 5s - loss: 2.5003e-05 - 5s/epoch - 5ms/step
Epoch 7/50
1021/1021 - 5s - loss: 6.8556e-05 - 5s/epoch - 4ms/step
Epoch 8/50
1021/1021 - 5s - loss: 3.6949e-05 - 5s/epoch - 5ms/step
Epoch 9/50
1021/1021 - 5s - loss: 2.4604e-05 - 5s/epoch - 5ms/step
Epoch 10/50
1021/1021 - 5s - loss: 4.8684e-05 - 5s/epoch - 5ms/step
Epoch 11/50
1021/1021 - 5s - loss: 4.3705e-05 - 5s/epoch - 5ms/step
Epoch 12/50
1021/1021 - 5s - loss: 3.1995e-05 - 5s/epoch - 5ms/step
Epoch 13/50
1021/1021 - 5s - loss: 2.0662e-05 - 5s/epoch - 5ms/step
Epoch 14/50
1021/1021 - 6s - loss: 6.9764e-05 - 6s/epoch - 5ms/step
Epoch 15/50
1021/1021 - 5s - loss: 5.4982e-05 - 5s/epoch 

In [10]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

16/16 [==============================] - 0s 5ms/step


In [11]:
train_predict = scaler.inverse_transform(train_predict)
train = scaler.inverse_transform(train)
test_predict = scaler.inverse_transform(test_predict)
test = scaler.inverse_transform(test)


In [12]:
"""train_rmse = np.sqrt(np.mean(np.power((train - train_predict), 2)))
test_rmse = np.sqrt(np.mean(np.power((test - test_predict), 2)))
print('Train RMSE: %.2f' % train_rmse)
print('Test RMSE: %.2f' % test_rmse)"""

from sklearn.metrics import r2_score
import math
from sklearn.metrics import mean_squared_error
train_rmse = np.sqrt(mean_squared_error(train, train_predict))
test_rmse = np.sqrt(mean_squared_error(test, test_predict))
print('R2 Score_train:', r2_score(train, train_predict))
print('R2 Score_test:', r2_score(test, test_predict))

print('Train RMSE: %.2f' % train_rmse)
print('Test RMSE: %.2f' % test_rmse)

R2 Score_train: 0.9998631431066463
R2 Score_test: 0.9998767976213309
Train RMSE: 0.35
Test RMSE: 0.34


In [13]:
df=pd.DataFrame({'Actual':train.tolist(),'Predicted from lstm':train_predict.tolist()})
df.to_excel('lstm_10.xlsx')
df

,Actual,Predicted from lstm
0,[123.4699935913086],[123.09784698486328]
1,[160.00999450683594],[159.91668701171875]
2,[133.6799774169922],[133.35665893554688]
3,[126.95999908447266],[126.60183715820312]
4,[142.25997924804688],[141.9957275390625]
...,...,...
1016,[72.94999694824219],[72.72300720214844]
1017,[76.41999816894531],[76.16056823730469]
1018,[82.36000061035156],[82.05317687988281]
1019,[50.45000076293945],[50.52241897583008]


In [14]:
df=pd.DataFrame({'Actual':test.tolist(),'Predicted from lstm':test_predict.tolist()})
df.to_excel('lstm_10_test.xlsx')
df

,Actual,Predicted from lstm
0,[97.32999420166016],[96.94790649414062]
1,[96.66000366210938],[96.27996063232422]
2,[109.61000061035156],[109.21112823486328]
3,[109.21998596191406],[108.82106018066406]
4,[120.69998168945312],[120.31877899169922]
...,...,...
499,[111.66000366210938],[111.26210021972656]
500,[136.57000732421875],[136.2647705078125]
501,[244.66000366210938],[246.08694458007812]
502,[109.56999206542969],[109.1711196899414]
